In [2]:
import pandas as pd

In [4]:
#读取附件2 企业信息
data1=pd.read_excel('附件2：302家无信贷记录企业的相关数据.xlsx',sheet_name='企业信息')
data1.head()

,企业代号,企业名称
0,E124,个体经营E124
1,E125,个体经营E125
2,E126,个体经营E126
3,E127,个体经营E127
4,E128,个体经营E128


In [5]:
#读取附件1 进项发票信息
data2=pd.read_excel('附件2：302家无信贷记录企业的相关数据.xlsx',sheet_name='进项发票信息')
data2.head()

,企业代号,发票号码,开票日期,销方单位代号,金额,税额,价税合计,发票状态,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,E124,18891676,2017-09-01 00:00:00.000000,C00014,338.46,57.54,396.0,有效发票,NaN,NaN,NaN
1,E124,18691267,2017-09-01 00:00:00.000000,C00480,230.10,6.90,237.0,有效发票,NaN,NaN,NaN
2,E124,12995412,2017-09-01 00:00:00.000000,C23675,223.30,6.70,230.0,有效发票,NaN,NaN,NaN
3,E124,6378193,2017-09-01 11:11:02.999996,C00333,90090.09,9909.91,100000.0,有效发票,NaN,NaN,NaN
4,E124,6378194,2017-09-01 11:12:00.999997,C00333,90090.09,9909.91,100000.0,有效发票,NaN,NaN,NaN


In [6]:
#读取附件1 销项发票信息
data3=pd.read_excel('附件2：302家无信贷记录企业的相关数据.xlsx',sheet_name='销项发票信息')
data3.head()

,企业代号,发票号码,开票日期,购方单位代号,金额,税额,价税合计,发票状态
0,E124,15212483,2017-09-01 11:58:43.000000,D00585,839350.55,92328.56,931679.11,有效发票
1,E124,15212484,2017-09-01 11:59:19.999997,D00585,900900.90,99099.10,1000000.00,有效发票
2,E124,15212485,2017-09-01 11:59:50.999997,D00585,900900.90,99099.10,1000000.00,有效发票
3,E124,15212486,2017-09-01 12:00:44.999997,D00585,697365.74,76710.23,774075.97,有效发票
4,E124,6089615,2017-09-05 10:58:38.000004,D00108,726216.22,79883.78,806100.00,有效发票


In [7]:
def filter_extreme_MAD(dataframe,n=3):
    # MAD:绝对中位差去极值
    for i in dataframe.columns:
        median = dataframe[i].quantile(0.5)
        new_median = ((dataframe[i] - median).abs()).quantile(0.50)
        max_range = median + n * new_median
        min_range = median - n * new_median
        dataframe[i]=pd.DataFrame(np.clip(dataframe[i].values, min_range, max_range),columns=None)

    return dataframe
def filter_extreme_3sigma(dataframe,n=3):
    # 3sigma 去极值
    for i in dataframe.columns:
        mean=dataframe[i].mean()
        std=dataframe[i].std()
        max_range=mean+n*std
        min_range=mean-n*std
        dataframe[i] = pd.DataFrame(np.clip(dataframe[i].values, min_range, max_range), columns=None)
    return dataframe

def filter_extreme_percentile(dataframe,min=0.025,max=0.975):
    # 百分位法 去极值
    for i in dataframe.columns:
        Temp=dataframe[i].sort_values()
        q=Temp.quantile([min,max])
        dataframe[i] = pd.DataFrame(np.clip(dataframe[i].values, q.iloc[0], q.iloc[1]), columns=None)
    return dataframe

In [8]:
#对data2进行异常值与缺失值处理
for i in data2.columns:
    #对数据进行异常值处理，若报错（即数据为文本或者非定量变量），则自动跳过
    try:
        data2[i]=filter_extreme_3sigma(data2[i],n=3)
    except:
        pass
    
#对data3进行异常值与缺失值处理
for i in data3.columns:
    #对数据进行异常值处理，若报错（即数据为文本或者非定量变量），则自动跳过
    try:
        data3[i]=filter_extreme_3sigma(data3[i],n=3)
    except:
        pass        

In [9]:
#以企业代号为分组项，对'金额', '税额', '价税合计'这三个变量提取计数、均值、标准差、最小值、1/4位数、1/2位数、3/4位数，最大值构造特征
#可以看到，从3个变量衍生成了24个变量
data2_huizng_0=data2.groupby('企业代号')[['金额', '税额', '价税合计']].describe()
data2_huizng_0.head()

金额                                                               \
        count           mean            std        min       25%       50%   
企业代号                                                                         
E124  17411.0   52971.025771  124108.978850 -881892.24   530.970   9999.03   
E125  20288.0   54017.359583  122409.037582 -970873.79   640.150  13643.74   
E126    533.0  220126.048780  300449.260954  -95774.37  6767.000  86150.86   
E127   1450.0    1194.927276    9323.027176    -513.27   172.745    286.74   
E128   3222.0    3197.470261   15130.191636  -92307.69   158.245    283.02   

                                    税额                ...              \
              75%         max    count          mean  ...         75%   
企业代号                                                  ...               
E124   90909.0900  5504587.16  17411.0   4014.860334  ...   5589.2250   
E125   91743.1200  5504587.16  20288.0   4037.314564  ...   5937.3650   
E126  293473.1800   999961.31    533.0  31292.976811  ...  36856.8600   
E127     549.2025   283018.87   1450.0     84.226476  ...     47.9425   
E128     770.6050   359223.30   3222.0    150.773029  ...     55.1800   

                    价税合计                                                      \
            max    count           mean            std        min        25%   
企业代号                                                                           
E124  495412.84  17411.0   56985.886105  130973.886146 -1022995.0   580.0000   
E125  495412.84  20288.0   58054.674146  128906.305832 -1022995.0   683.0750   
E126  167396.16    533.0  251419.025591  344080.546899   -98647.6  6970.0000   
E127   29931.03   1450.0    1279.153752   10137.630575     -580.0   200.0000   
E128   15692.31   3222.0    3348.243290   15783.480017  -108000.0   173.3025   

                                      
           50%        75%        max  
企业代号                                  
E124  11576.11  100000.00  6000000.0  
E125  14850.00  100000.00  6000000.0  
E126  99600.00  340428.89  1159955.1  
E127    320.00     599.75   300000.0  
E128    319.50     800.00   370000.0  

[5 rows x 24 columns]

In [10]:
#根据发票状态变量，可以衍生出特征，发票状态个数，去重值，频数最高的发票状态类别，频数最高的发票状态类别对应频数
#可以看到，从3个变量衍生成了24个变量
data2_huizng_1=data2.groupby('企业代号')[['发票状态']].describe()
data2_huizng_1

发票状态                    
      count unique   top   freq
企业代号                           
E124  17411      2  有效发票  15264
E125  20288      2  有效发票  17776
E126    533      2  有效发票    515
E127   1450      2  有效发票   1424
E128   3222      2  有效发票   3134
...     ...    ...   ...    ...
E421     19      1  有效发票     19
E422      3      1  有效发票      3
E423     21      1  有效发票     21
E424     25      1  有效发票     25
E425    118      2  有效发票    116

[302 rows x 4 columns]

In [11]:
#将构造好的两个特征列表合并为data2_huizng
data2_huizng=pd.concat([data2_huizng_0,data2_huizng_1] ,axis=1)
data2_huizng

金额                                                                \
        count           mean            std        min       25%        50%   
企业代号                                                                          
E124  17411.0   52971.025771  124108.978850 -881892.24   530.970   9999.030   
E125  20288.0   54017.359583  122409.037582 -970873.79   640.150  13643.740   
E126    533.0  220126.048780  300449.260954  -95774.37  6767.000  86150.860   
E127   1450.0    1194.927276    9323.027176    -513.27   172.745    286.740   
E128   3222.0    3197.470261   15130.191636  -92307.69   158.245    283.020   
...       ...            ...            ...        ...       ...        ...   
E421     19.0    1599.674211    1349.218879     223.30   283.010    943.400   
E422      3.0     296.463333     127.059975     188.68   226.415    264.150   
E423     21.0    2906.959048    4305.431175   -8615.38   806.210   1456.310   
E424     25.0   10173.196000   15129.873082      64.65   671.890   1283.020   
E425    118.0     504.402712    1075.171851    -264.15    86.235    176.725   

                                    税额                ...           价税合计  \
              75%         max    count          mean  ...            std   
企业代号                                                  ...                  
E124   90909.0900  5504587.16  17411.0   4014.860334  ...  130973.886146   
E125   91743.1200  5504587.16  20288.0   4037.314564  ...  128906.305832   
E126  293473.1800   999961.31    533.0  31292.976811  ...  344080.546899   
E127     549.2025   283018.87   1450.0     84.226476  ...   10137.630575   
E128     770.6050   359223.30   3222.0    150.773029  ...   15783.480017   
...           ...         ...      ...           ...  ...            ...   
E421    2830.1900     3773.58     19.0     93.483684  ...    1432.100449   
E422     350.3550      436.56      3.0     23.536667  ...     144.222051   
E423    6335.0400     9700.85     21.0    381.217143  ...    4983.621734   
E424   15461.7300    63309.06     25.0    706.779600  ...   16125.413400   
E425     360.6075     9230.77    118.0     44.412881  ...    1215.733825   

                                                             发票状态         \
            min        25%       50%        75%        max  count unique   
企业代号                                                                       
E124 -1022995.0   580.0000  11576.11  100000.00  6000000.0  17411      2   
E125 -1022995.0   683.0750  14850.00  100000.00  6000000.0  20288      2   
E126   -98647.6  6970.0000  99600.00  340428.89  1159955.1    533      2   
E127     -580.0   200.0000    320.00     599.75   300000.0   1450      2   
E128  -108000.0   173.3025    319.50     800.00   370000.0   3222      2   
...         ...        ...       ...        ...        ...    ...    ...   
E421      230.0   300.0000   1000.00    3000.00     4000.0     19      1   
E422      200.0   240.0000    280.00     380.00      480.0      3      1   
E423   -10080.0   830.4000   1500.00    7412.00    11350.0     21      1   
E424       75.0   712.2000   1460.00   16389.43    67107.6     25      1   
E425     -280.0    91.6000    189.00     396.75    10800.0    118      2   

                   
       top   freq  
企业代号               
E124  有效发票  15264  
E125  有效发票  17776  
E126  有效发票    515  
E127  有效发票   1424  
E128  有效发票   3134  
...    ...    ...  
E421  有效发票     19  
E422  有效发票      3  
E423  有效发票     21  
E424  有效发票     25  
E425  有效发票    116  

[302 rows x 28 columns]

In [12]:
#将data2_huizng与data1进行合并，以企业代号为连接键
union_data=pd.merge(data1,data2_huizng,left_on='企业代号',right_on='企业代号')
union_data

C:\Users\Administrator\AppData\Roaming\Python\Python37\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


,企业代号,企业名称,"(金额, count)","(金额, mean)","(金额, std)","(金额, min)","(金额, 25%)","(金额, 50%)","(金额, 75%)","(金额, max)",...,"(价税合计, std)","(价税合计, min)","(价税合计, 25%)","(价税合计, 50%)","(价税合计, 75%)","(价税合计, max)","(发票状态, count)","(发票状态, unique)","(发票状态, top)","(发票状态, freq)"
0,E124,个体经营E124,17411.0,52971.025771,124108.978850,-881892.24,530.970,9999.030,90909.0900,5504587.16,...,130973.886146,-1022995.0,580.0000,11576.11,100000.00,6000000.0,17411,2,有效发票,15264
1,E125,个体经营E125,20288.0,54017.359583,122409.037582,-970873.79,640.150,13643.740,91743.1200,5504587.16,...,128906.305832,-1022995.0,683.0750,14850.00,100000.00,6000000.0,20288,2,有效发票,17776
2,E126,个体经营E126,533.0,220126.048780,300449.260954,-95774.37,6767.000,86150.860,293473.1800,999961.31,...,344080.546899,-98647.6,6970.0000,99600.00,340428.89,1159955.1,533,2,有效发票,515
3,E127,个体经营E127,1450.0,1194.927276,9323.027176,-513.27,172.745,286.740,549.2025,283018.87,...,10137.630575,-580.0,200.0000,320.00,599.75,300000.0,1450,2,有效发票,1424
4,E128,个体经营E128,3222.0,3197.470261,15130.191636,-92307.69,158.245,283.020,770.6050,359223.30,...,15783.480017,-108000.0,173.3025,319.50,800.00,370000.0,3222,2,有效发票,3134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,E421,***保温材料有限公司,19.0,1599.674211,1349.218879,223.30,283.010,943.400,2830.1900,3773.58,...,1432.100449,230.0,300.0000,1000.00,3000.00,4000.0,19,1,有效发票,19
298,E422,***童装店,3.0,296.463333,127.059975,188.68,226.415,264.150,350.3550,436.56,...,144.222051,200.0,240.0000,280.00,380.00,480.0,3,1,有效发票,3
299,E423,***通风设备有限公司,21.0,2906.959048,4305.431175,-8615.38,806.210,1456.310,6335.0400,9700.85,...,4983.621734,-10080.0,830.4000,1500.00,7412.00,11350.0,21,1,有效发票,21
300,E424,***贸易有限公司,25.0,10173.196000,15129.873082,64.65,671.890,1283.020,15461.7300,63309.06,...,16125.413400,75.0,712.2000,1460.00,16389.43,67107.6,25,1,有效发票,25


In [13]:
#同理处理data3

data3_huizng=data3.groupby('企业代号')[['金额', '税额', '价税合计']].describe()

data3_huizng=pd.concat([data3_huizng,data3.groupby('企业代号')[['发票状态']].describe()] ,axis=1)

#将union_data与data3_huizng合并
union_data=pd.merge(union_data,data3_huizng,left_on='企业代号',right_on='企业代号')
union_data.shape

C:\Users\Administrator\AppData\Roaming\Python\Python37\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


(302, 58)

In [24]:
#因为附件2没有信誉评级，无需剔除
# del union_data['信誉评级']

In [17]:
#可以看到，一共有302家企业数据，59-3项特征（X）,Y是【信贷等级】
union_data.head()

,企业代号,企业名称,"('金额', 'count')_x","('金额', 'mean')_x","('金额', 'std')_x","('金额', 'min')_x","('金额', '25%')_x","('金额', '50%')_x","('金额', '75%')_x","('金额', 'max')_x",...,"(价税合计, std)","(价税合计, min)","(价税合计, 25%)","(价税合计, 50%)","(价税合计, 75%)","(价税合计, max)","(发票状态, count)","(发票状态, unique)","(发票状态, top)","(发票状态, freq)"
0,E124,个体经营E124,17411.0,52971.025771,124108.978850,-881892.24,530.970,9999.03,90909.0900,5504587.16,...,415753.233732,-1000000.0,527509.96,1000000.00,1000000.000,1109999.99,1293,2,有效发票,1099
1,E125,个体经营E125,20288.0,54017.359583,122409.037582,-970873.79,640.150,13643.74,91743.1200,5504587.16,...,400992.461740,-1000000.0,519571.52,1000000.00,1000000.000,1109999.99,1595,2,有效发票,1380
2,E126,个体经营E126,533.0,220126.048780,300449.260954,-95774.37,6767.000,86150.86,293473.1800,999961.31,...,408119.411951,-1016304.0,71162.00,322403.13,881583.225,1082430.50,1516,2,有效发票,1264
3,E127,个体经营E127,1450.0,1194.927276,9323.027176,-513.27,172.745,286.74,549.2025,283018.87,...,249002.208519,-900000.0,100000.00,102900.00,102999.000,1029999.99,4026,2,有效发票,3904
4,E128,个体经营E128,3222.0,3197.470261,15130.191636,-92307.69,158.245,283.02,770.6050,359223.30,...,292912.269757,0.0,99500.00,99990.00,100000.000,1000000.00,1255,2,有效发票,1161


In [18]:
#将附件1数据保存下来
union_data.to_csv('cleandata附件2.csv',index=None,encoding='gbk')